In [ ]:
%pip install openai
import streamlit as st
import pandas as pd
import openai

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import os
openai.api_key = st.secrets["OPENAI_API_KEY"]


In [13]:
# Load CSVs
import pandas as pd

base_df = pd.read_csv("Base.csv")
apparel_df = pd.read_csv("Apparel.csv")
construction_df = pd.read_csv("Construction.csv")
energy_df = pd.read_csv("Energy.csv")
hospitality_df = pd.read_csv("Hospitality.csv")
transportation_df = pd.read_csv("Transportation.csv")

# Dictionary linking industry names to their DataFrames
industry_dfs = {
    "Apparel": apparel_df,
    "Construction": construction_df,
    "Energy": energy_df,
    "Hospitality": hospitality_df,
    "Transportation": transportation_df
}


In [14]:
#Check the loaded dataset heads
print("Base:")
print(base_df.head(), "\n")

print("Apparel:")
print(apparel_df.head(), "\n")

print("Construction:")
print(construction_df.head(), "\n")

print("Energy:")
print(energy_df.head(), "\n")

print("Hospitality:")
print(hospitality_df.head(), "\n")

print("Transportation:")
print(transportation_df.head(), "\n")


Base:
               Base Name            Base Code Payment Terms  \
0       Loop Calibrators             CAL-0002         Net 5   
1     Keyboard UK Layout           KEYBOARDUK         Net 5   
2       Desktop Computer      DESKTOPCOMPUTER         Net 5   
3  Home Security Console  HOMESECURITYCONSOLE         Net 5   
4        Doorbell Camera       DOORBELLCAMERA         Net 5   

   Minimum Order Quantity  
0                       1  
1                       1  
2                       1  
3                       1  
4                       1   

Apparel:
   Name - Apparel Industry Code - Apparel Industry Payment Terms  \
0       Loop Calibrators-A              CAL-0002-A        Net 10   
1     Keyboard UK Layout-A            KEYBOARDUK-A        Net 10   
2       Desktop Computer-A       DESKTOPCOMPUTER-A        Net 10   
3  Home Security Console-A   HOMESECURITYCONSOLE-A        Net 10   
4        Doorbell Camera-A        DOORBELLCAMERA-A        Net 10   

   Minimum Order Quantity  

In [ ]:
def get_recommendation(product_name, industry):
    # Find the base product details
    base_product = base_df[base_df["Base Name"] == product_name]
    if base_product.empty:
        return "Product not found in base data."

    # Default to base product info
    reco_product = product_name
    reco_code = base_product["Base Code"].values[0]
    moq = int(base_product["Minimum Order Quantity"].values[0])
    terms = int(base_product["Payment Terms"].values[0])

    # Check for industry-specific match
    if industry in industry_dfs:
        df = industry_dfs[industry]
        match = df[df[df.columns[0]].str.startswith(product_name)]
        if not match.empty:
            reco_product = match[df.columns[0]].values[0]
            reco_code = match[df.columns[1]].values[0]
            moq = int(match["Minimum Order Quantity"].values[0])
            terms = int(match["Payment Terms"].values[0])

    return {
        "Product": product_name,
        "Industry": industry,
        "Recommended Product": reco_product,
        "Recommended Code": reco_code,
        "MOQ": moq,
        "Payment Terms": terms
    }


In [16]:
def get_deal_insights(product, industry, moq, payment_terms):
    prompt = f"""
    Product: {product}
    Industry: {industry}
    Recommended Product: {product}-{industry[0].upper()}
    Minimum Order Quantity: {moq}
    Payment Terms: {payment_terms}

    Based on this context, answer the following:

    1. What is the probability (0-100%) of closing this deal?
    2. What is the profitability rating? (Low / Medium / High)
    3. What should be the next best step for the sales rep to close this deal?

    Format the response as:
    Deal Probability: <percent>
    Profitability: <rating>
    Next Step: <action>
    """

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [17]:
def parse_insights(response_text):
    lines = response_text.splitlines()
    deal_prob = next((line.split(":")[1].strip() for line in lines if "Deal Probability" in line), "")
    profit = next((line.split(":")[1].strip() for line in lines if "Profitability" in line), "")
    step = next((line.split(":")[1].strip() for line in lines if "Next Step" in line), "")
    return pd.Series([deal_prob, profit, step], index=["Deal Probability", "Profitability", "Next Step"])


In [18]:
# Example test
product_input = "Loop Calibrators"
industry_input = "Apparel"

rec = get_recommendation(product_input, industry_input)
insight_text = get_deal_insights(rec["Product"], rec["Industry"], rec["MOQ"], rec["Payment Terms"])
insight_parsed = parse_insights(insight_text)

# Combine everything
rec.update(insight_parsed.to_dict())
print(rec)


{'Product': 'Loop Calibrators', 'Industry': 'Apparel', 'Recommended Product': 'Loop Calibrators-A', 'Recommended Code': 'CAL-0002-A', 'MOQ': 10, 'Payment Terms': 'Net 10', 'Deal Probability': '80%', 'Profitability': 'Medium', 'Next Step': 'Follow up with the customer to discuss payment terms and confirm the order quantity.'}
